In [1]:
from supabase import create_client
import pandas as pd

SUPABASE_URL = "https://qmjpxafyxikhynczflch.supabase.co"
SUPABASE_KEY = "sb_publishable_3yT3_nQMXBfLu27TyCU9UQ_jI57jX4q"

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Load your dataset
response = supabase.table("air_quality_raw") \
    .select("*") \
    .order("date") \
    .execute()

df = pd.DataFrame(response.data)


# Ensure the 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Get the last date in the dataset
last_date = df['date'].max()

# Calculate the date 30 days before the last date
start_date = last_date - pd.Timedelta(days=30)

# Filter the dataframe for the last 30 days, excluding today
last_30_days_df = df[(df['date'] > start_date) & (df['date'] < last_date)]

# Save the filtered data to a new CSV
last_30_days_df.to_csv('last_30_days_data.csv', index=False)

print("Filtered data saved to 'last_30_days_data.csv'")

C:\Users\Vishwas vinayakumar\AppData\Roaming\Python\Python313\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Filtered data saved to 'last_30_days_data.csv'


In [3]:
df.columns = df.columns.str.strip()
print(df.head(30))

                                      id                      date  pm25  \
0   36429414-2cee-4d94-a8e7-c5868cdcbd5f 2025-10-01 00:00:00+00:00    68   
1   199969b7-d3d8-4153-b61d-93fbc9a4a1cf 2025-10-02 00:00:00+00:00    71   
2   677b7680-2d7a-4fa7-81e3-f23abdb31c03 2025-10-03 00:00:00+00:00    69   
3   d47f9dda-1a3d-4855-89fe-3ba6b1918bc1 2025-10-04 00:00:00+00:00    50   
4   52454375-ba21-41c1-880c-8f71b38e4a9d 2025-10-05 00:00:00+00:00    28   
5   f25f92dc-c0ef-4bc9-800d-bc63913aef97 2025-10-06 00:00:00+00:00    37   
6   cb0d64dc-56ac-48ae-9c29-b96432cbc089 2025-10-07 00:00:00+00:00    48   
7   6c3cc57f-609c-46da-a3b7-1fcff7689fda 2025-10-08 00:00:00+00:00    46   
8   ecd1e7d7-0a56-4220-87b1-bc9077f6c37b 2025-10-09 00:00:00+00:00    38   
9   6a9f02fa-6665-4b7c-b111-2d9b246cb980 2025-10-10 00:00:00+00:00    44   
10  f38cde04-f081-4ad1-abb5-94bf51e72585 2025-10-11 00:00:00+00:00    41   
11  8ab142c2-097a-42b2-acf8-251dc8106ec7 2025-10-12 00:00:00+00:00    37   
12  1a18ae53

In [4]:
cols = ['pm25', 'pm10', 'no2', 'o3']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          31 non-null     object             
 1   date        31 non-null     datetime64[ns, UTC]
 2   pm25        31 non-null     int64              
 3   pm10        31 non-null     int64              
 4   no2         31 non-null     int64              
 5   o3          31 non-null     int64              
 6   created_at  31 non-null     object             
dtypes: datetime64[ns, UTC](1), int64(4), object(2)
memory usage: 1.8+ KB
None


In [5]:
# Convert Date and Set Index
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date').reset_index(drop=True)

# interprolate values (linear)
df[['pm25', 'pm10', 'no2', 'o3']] = df[['pm25', 'pm10', 'no2', 'o3']].interpolate(method='linear')

# Fill remaining NaN at start/end using forword fill then backword fill
df[['pm25', 'pm10', 'no2', 'o3']] =df[['pm25', 'pm10', 'no2', 'o3']].ffill().bfill()

df.set_index('date', inplace=True)
print(df.info())
print(df.head(30))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 31 entries, 2025-10-01 00:00:00+00:00 to 2026-01-10 00:00:00+00:00
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          31 non-null     object
 1   pm25        31 non-null     int64 
 2   pm10        31 non-null     int64 
 3   no2         31 non-null     int64 
 4   o3          31 non-null     int64 
 5   created_at  31 non-null     object
dtypes: int64(4), object(2)
memory usage: 1.7+ KB
None
                                                             id  pm25  pm10  \
date                                                                          
2025-10-01 00:00:00+00:00  36429414-2cee-4d94-a8e7-c5868cdcbd5f    68    32   
2025-10-02 00:00:00+00:00  199969b7-d3d8-4153-b61d-93fbc9a4a1cf    71    27   
2025-10-03 00:00:00+00:00  677b7680-2d7a-4fa7-81e3-f23abdb31c03    69    18   
2025-10-04 00:00:00+00:00  d47f9dda-1a3d-4855-89fe-3ba6b1918bc1    50    10   

In [6]:
missing_cols = df.columns[df.isnull().any()]
print("Columns with Missing Values:")
print(missing_cols)

Columns with Missing Values:
Index([], dtype='object')


In [7]:
print(df[df.isna().any(axis=1)])

Empty DataFrame
Columns: [id, pm25, pm10, no2, o3, created_at]
Index: []


In [8]:
# Replace '-' with NaN and forward fill
df.replace('-', np.nan, inplace=True)
df.ffill(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 31 entries, 2025-10-01 00:00:00+00:00 to 2026-01-10 00:00:00+00:00
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          31 non-null     object
 1   pm25        31 non-null     int64 
 2   pm10        31 non-null     int64 
 3   no2         31 non-null     int64 
 4   o3          31 non-null     int64 
 5   created_at  31 non-null     object
dtypes: int64(4), object(2)
memory usage: 1.7+ KB


In [9]:
print(df.head(30))

                                                             id  pm25  pm10  \
date                                                                          
2025-10-01 00:00:00+00:00  36429414-2cee-4d94-a8e7-c5868cdcbd5f    68    32   
2025-10-02 00:00:00+00:00  199969b7-d3d8-4153-b61d-93fbc9a4a1cf    71    27   
2025-10-03 00:00:00+00:00  677b7680-2d7a-4fa7-81e3-f23abdb31c03    69    18   
2025-10-04 00:00:00+00:00  d47f9dda-1a3d-4855-89fe-3ba6b1918bc1    50    10   
2025-10-05 00:00:00+00:00  52454375-ba21-41c1-880c-8f71b38e4a9d    28    14   
2025-10-06 00:00:00+00:00  f25f92dc-c0ef-4bc9-800d-bc63913aef97    37    15   
2025-10-07 00:00:00+00:00  cb0d64dc-56ac-48ae-9c29-b96432cbc089    48    14   
2025-10-08 00:00:00+00:00  6c3cc57f-609c-46da-a3b7-1fcff7689fda    46    14   
2025-10-09 00:00:00+00:00  ecd1e7d7-0a56-4220-87b1-bc9077f6c37b    38    15   
2025-10-10 00:00:00+00:00  6a9f02fa-6665-4b7c-b111-2d9b246cb980    44    14   
2025-10-11 00:00:00+00:00  f38cde04-f081-4ad1-abb5-9

In [10]:
# Select Features (Pollutant Columns)
features = ['pm25', 'pm10', 'no2', 'o3']
data = df[features]

# Handle Missing Values (Using Simple Interpolation)
# This fills gaps using a linear trend between known values.
data = data.interpolate(method='linear')

# Verify no more missing data
print("\nMissing values after interpolation:")
print(data.isnull().sum())
print("\nProcessed Data Head:")
print(data.head())


Missing values after interpolation:
pm25    0
pm10    0
no2     0
o3      0
dtype: int64

Processed Data Head:
                           pm25  pm10  no2  o3
date                                          
2025-10-01 00:00:00+00:00    68    32   18  25
2025-10-02 00:00:00+00:00    71    27   15  26
2025-10-03 00:00:00+00:00    69    18    9  28
2025-10-04 00:00:00+00:00    50    10    6  23
2025-10-05 00:00:00+00:00    28    14    9  25


In [11]:
# Scaling and Sequence Generation

# Scaling
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Define Sequence Parameters
N_IN = 7      # Lookback window (7 days)
N_OUT = 2     # Prediction horizon (2 days)
N_FEATURES = len(features) # 4 pollutants

# Function to Create Sequences (Sliding Window)
def create_sequences(data, n_in, n_out):
    X, y = [], []
    # Loop from the starting point up to the end minus the total length of the sequence
    for i in range(len(data) - n_in - n_out + 1):
        # Input sequence (7 days)
        end_ix = i + n_in
        X.append(data[i:end_ix, :])

        # Output sequence (2 days starting right after the input ends)
        out_ix = end_ix + n_out
        y.append(data[end_ix:out_ix, :])

    return np.array(X), np.array(y)

# Create the sequences
X, y = create_sequences(scaled_data, N_IN, N_OUT)

print(f"\nShape of Input (X): (Samples, Lookback Days, Features) -> {X.shape}")
print(f"Shape of Output (y): (Samples, Prediction Horizon, Features) -> {y.shape}")

# Chronological Train/Test Split (80% Train, 20% Test)
# Time series data must be split chronologically.
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"Train/Test Split: {len(X_train)} training samples, {len(X_test)} test samples.")


Shape of Input (X): (Samples, Lookback Days, Features) -> (23, 7, 4)
Shape of Output (y): (Samples, Prediction Horizon, Features) -> (23, 2, 4)
Train/Test Split: 18 training samples, 5 test samples.


In [12]:
# Build, Train, and Evaluate the LSTM Model

#  Build the Model
model = Sequential()

# LSTM Layer: 50 units, returns sequences for the next layer
model.add(LSTM(50, activation='relu', input_shape=(N_IN, N_FEATURES)))

# Output Layer: Dense layer to predict the 4 features (pollutants) for the 2 days (N_OUT)
# flatten the output (N_OUT * N_FEATURES = 8 values)
model.add(Dense(N_OUT * N_FEATURES))
model.compile(optimizer='adam', loss='mse')

# Display Model Summary
print("\nModel Summary:")
model.summary()

# Training the Model
# EarlyStopping prevents overfitting by stopping training if validation loss doesn't improve.
callback = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

history = model.fit(X_train, y_train.reshape(X_train.shape[0], -1), # Reshape y_train for the model
                    epochs=100,
                    batch_size=32,
                    verbose=1,
                    validation_split=0.1, # Use 10% of training data for validation
                    callbacks=[callback])

# Evaluate Model on the Test Set
test_loss = model.evaluate(X_test, y_test.reshape(X_test.shape[0], -1), verbose=0)
print(f"\nTest Set Mean Squared Error (MSE): {test_loss:.4f}")

# Make Predictions on Test Set
y_pred_scaled = model.predict(X_test)

# Reshape predictions and actuals back to (Samples, N_OUT, N_FEATURES)
y_pred_scaled = y_pred_scaled.reshape(y_test.shape)

# Inverse Transform (Rescale to original values)
# To inverse transform (must first flatten the 3D arrays to 2D)
y_test_original = scaler.inverse_transform(y_test.reshape(-1, N_FEATURES))
y_pred_original = scaler.inverse_transform(y_pred_scaled.reshape(-1, N_FEATURES))

# Reshape back to 3D for comparison plots
y_test_original = y_test_original.reshape(y_test.shape)
y_pred_original = y_pred_original.reshape(y_test.shape)

C:\Users\Vishwas vinayakumar\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        11,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │           408 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,408 (44.56 KB)

 Trainable params: 11,408 (44.56 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.2036 - val_loss: 0.3370
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.1979 - val_loss: 0.3288
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 0.1921 - val_loss: 0.3206
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.1864 - val_loss: 0.3124
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.1807 - val_loss: 0.3041
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 0.1750 - val_loss: 0.2958
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.1693 - val_loss: 0.2876
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - loss: 0.1636 - val_loss: 0.2792
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.1578 - val_loss: 0.2709
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - loss: 0.1521 - val_loss: 0.2625
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.1463 - val_loss: 0.2539
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.1404 - val_

In [13]:
# Final 2-Day Forecast

# Prepare the Final Input
# Take the last N_IN (7) days from the scaled dataset
last_7_days_scaled = scaled_data[-N_IN:]

# Reshape the input to match model's expected input shape: (1, N_IN, N_FEATURES)
X_input = last_7_days_scaled.reshape(1, N_IN, N_FEATURES)

# Generate the Forecast
forecast_scaled = model.predict(X_input)

# Reshape the forecast (1, N_OUT * N_FEATURES) to (N_OUT, N_FEATURES) for inverse scaling
forecast_scaled = forecast_scaled.reshape(N_OUT, N_FEATURES)

# Inverse Transform the Forecast to Original Units
final_forecast_original = scaler.inverse_transform(forecast_scaled)

# Create a Forecast DataFrame
last_date = df.index[-1]
forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=N_OUT, freq='D')

forecast_df = pd.DataFrame(final_forecast_original,
                           index=forecast_dates,
                           columns=features)


print("\n\n#####################################################")
print("## ✅ 2-DAY AIR QUALITY FORECAST (NEXT 48 HOURS) ✅ ##")
print("#####################################################")
print(forecast_df)
print("-----------------------------------------------------")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step


#####################################################
## ✅ 2-DAY AIR QUALITY FORECAST (NEXT 48 HOURS) ✅ ##
#####################################################
                                pm25       pm10        no2         o3
2026-01-11 00:00:00+00:00  56.843090  20.436214  11.661908  28.735723
2026-01-12 00:00:00+00:00  56.174904  15.636422  12.765058  30.317410
-----------------------------------------------------


In [14]:
from datetime import timedelta

last_known_date = df.index.max()
 # from training data

forecast_df = forecast_df.copy()
forecast_df["forecast_date"] = [
    last_known_date + timedelta(days=i + 1)
    for i in range(len(forecast_df))
]


In [15]:
# AQI CALCULATION (POST-PROCESSING, NO MODEL CHANGE) ---

# AQI breakpoint tables (CPCB – India)
aqi_breakpoints = {
    "pm25": [(0,30,0,50),(31,60,51,100),(61,90,101,200),(91,120,201,300),(121,250,301,400),(251,500,401,500)],
    "pm10": [(0,50,0,50),(51,100,51,100),(101,250,101,200),(251,350,201,300),(351,430,301,400),(431,600,401,500)],
    "no2":  [(0,40,0,50),(41,80,51,100),(81,180,101,200),(181,280,201,300),(281,400,301,400),(401,1000,401,500)],
    "o3":   [(0,50,0,50),(51,100,51,100),(101,168,101,200),(169,208,201,300),(209,748,301,400)]
}

def calculate_sub_aqi(pollutant, value):
    for bp_lo, bp_hi, i_lo, i_hi in aqi_breakpoints[pollutant]:
        if bp_lo <= value <= bp_hi:
            return ((i_hi - i_lo)/(bp_hi - bp_lo)) * (value - bp_lo) + i_lo
    return aqi_breakpoints[pollutant][-1][3]  # max AQI for that pollutant


# Calculate AQI for each predicted day
aqi_values = []
aqi_categories = []

for _, row in forecast_df.iterrows():
    sub_indices = [
        calculate_sub_aqi('pm25', row['pm25']),
        calculate_sub_aqi('pm10', row['pm10']),
        calculate_sub_aqi('no2', row['no2']),
        calculate_sub_aqi('o3', row['o3'])
    ]
    final_aqi = int(max(sub_indices))
    aqi_values.append(final_aqi)

    if final_aqi <= 50:
        aqi_categories.append("Good")
    elif final_aqi <= 100:
        aqi_categories.append("Satisfactory")
    elif final_aqi <= 200:
        aqi_categories.append("Moderate")
    elif final_aqi <= 300:
        aqi_categories.append("Poor")
    elif final_aqi <= 400:
        aqi_categories.append("Very Poor")
    else:
        aqi_categories.append("Severe")

# Add AQI to forecast table
forecast_df["AQI"] = aqi_values
forecast_df["AQI_Category"] = aqi_categories

print("## FINAL AQI FORECAST (NEXT 2 DAYS) ##")
print("#############################################")
print(forecast_df)
print("---------------------------------------------")


## FINAL AQI FORECAST (NEXT 2 DAYS) ##
#############################################
                                pm25       pm10        no2         o3  \
2026-01-11 00:00:00+00:00  56.843090  20.436214  11.661908  28.735723   
2026-01-12 00:00:00+00:00  56.174904  15.636422  12.765058  30.317410   

                                      forecast_date  AQI  AQI_Category  
2026-01-11 00:00:00+00:00 2026-01-11 00:00:00+00:00   94  Satisfactory  
2026-01-12 00:00:00+00:00 2026-01-12 00:00:00+00:00   93  Satisfactory  
---------------------------------------------


In [21]:
forecast_df = forecast_df.rename(columns={"forecast_date": "date"})

In [22]:
forecast_df["date"] = forecast_df["date"].astype(str)

In [25]:
forecast_records = forecast_df[[
    "date",
    "pm25",
    "pm10",
    "no2",
    "o3"
]].to_dict(orient="records")


In [26]:
# Append forecasted rows into raw air quality table
supabase.table("air_quality_raw").insert(forecast_records).execute()


APIResponse(data=[{'id': 'c824d198-8443-419a-b165-55b213d9849d', 'date': '2026-01-11T00:00:00+00:00', 'pm25': 56.84309005737305, 'pm10': 20.436214447021484, 'no2': 11.661908149719238, 'o3': 28.7357234954834, 'created_at': '2026-01-11T05:28:12.382775+00:00'}, {'id': 'eaa36153-6acd-4d9f-ac67-674633000f04', 'date': '2026-01-12T00:00:00+00:00', 'pm25': 56.174903869628906, 'pm10': 15.636422157287598, 'no2': 12.765057563781738, 'o3': 30.31740951538086, 'created_at': '2026-01-11T05:28:12.382775+00:00'}], count=None)

In [28]:
print(forecast_df.index[-1])

2026-01-12 00:00:00+00:00


In [30]:
records = []

for forecast_date, row in forecast_df.iterrows():
    records.append({
        "forecast_date": forecast_date.isoformat(),
        "pm25": float(row["pm25"]),
        "pm10": float(row["pm10"]),
        "no2": float(row["no2"]),
        "o3": float(row["o3"]),
        "aqi": int(row["AQI"]),
        "aqi_category": row["AQI_Category"]
    })

supabase.table("air_quality_forecast").insert(records).execute()

print("Pushed rows:", len(records))


Pushed rows: 2
